In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [2]:
# Constants

MODEL = "llama3.2"

In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [21]:
summarize("https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/tools-reference/llm-tool?view=azureml-api-2")

'### Summary of the LLM Tool in Azure Machine Learning Prompt Flow Website\n\nThe LLM tool in Azure Machine Learning prompt flow enables users to leverage large language models like OpenAI, Azure OpenAI Service, or Azure AI model inference API for natural language processing. The tool provides APIs for Completion and Chat, which facilitate interactive conversations with text-based inputs and responses.\n\n### Announcements\n\n* Note: The embedding option has been removed from the LLM tool API setting, and users can use an embedding API with the embedding tool instead.\n* Tip: To use Microsoft Entra ID auth type for Azure OpenAI connection, assign either the Cognitive Services OpenAI User or Cognitive Services OpenAI Contributor role to a user or user-assigned managed identity.\n\n### Key Features\n\n#### Connections\nUsers need to set up connections to provisioned resources in prompt flow, including name, API key, API type, and API version.\n\n#### Inputs (Text Completion)\nThe followi

In [19]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [22]:
display_summary("https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/tools-reference/llm-tool?view=azureml-api-2")

**Summary of LLM Tool in Azure Machine Learning**
=============================================

The LLM tool in Azure Machine Learning is a platform for using large language models (LLMs) to process natural language inputs. It provides two APIs:

*   **Completion**: Generates text based on provided prompts, using OpenAI's completion models.
*   **Chat**: Facilitates interactive conversations with text-based inputs and responses, using OpenAI's chat models.

The tool supports key-based authentication for Azure OpenAI connections and has specific requirements for resource group names and API keys.

**Key Features**
---------------

*   Supports widely used LLMs like OpenAI and Azure OpenAI Service
*   Provides text completion and chat APIs
*   Allows users to set up connections to OpenAI resources or serverless API endpoints
*   Configures large language model parameters, including temperature and top-p probabilities
*   Includes features for managing prompt and response inputs

**Prerequisites**
-----------------

To use the LLM tool, users need to:

*   Create OpenAI resources and sign in with their personal API key
*   Deploy models to serverless API endpoints using the Azure AI model inference API